In [ ]:
import pandas as pd
import csv

In [ ]:
# # 댓글 데이터들의 전처리 과정입니다. 전처리된 파일로 대체합니다.

# data1 = pd.read_csv('badwords/reply_data1.csv', sep='\t')

# data1 = data1.replace(0, 2)
# data1 = data1.replace(1, 0)
# data1 = data1.replace(2, 1)

# data1 = data1.rename(columns={'content':'comment'})

# data1[data1['lable'].isnull()]
# data1['lable'] = pd.to_numeric(data1['lable'], errors='coerce')
# data1 = data1.dropna(subset=['lable'])
# data1['lable'] = data1['lable'].astype(int)

# f = open('badwords/reply_data2.txt', 'r', encoding='UTF-8')

# data2_list = f.read()

# f.close()

# data2_list = ''.join(data2_list.split('|'))
# data2_list = data2_list.split('\n')

# lable = []
# for i in range(0, len(data2_list)):
#     lable.append(data2_list[i][-1])
#     data2_list[i] = data2_list[i][:-1]

# data2 = pd.DataFrame([x for x in zip(data2_list,lable)])

# data2.columns = ['comment', 'lable']

# data3 = pd.read_csv('badwords/reply_data3.txt', sep = "\t", engine='python', encoding = "utf-8")
# data3 = data3.drop(['id'], axis=1)

# data3 = data3.replace(1, 2)
# data3 = data3.replace(0, 1)
# data3 = data3.replace(2, 0)

# data3.columns = ['comment', 'lable']

# data4 = pd.read_csv('badwords/reply_data4.tsv', sep = "\t", engine='python', encoding = "utf-8")

# data4 = data4.drop(['contain_gender_bias', 'bias'], axis=1)
# data4 = data4.replace('hate', 1)
# data4 = data4.replace('none', 0)

# data4.columns = ['comment', 'lable']

# reply_data = pd.concat([data1, data2, data4])
# data3 = data3.sample(n=40000)

# reply_data = pd.concat([reply_data, data3])
# reply_data

# reply_data = reply_data.replace('offensive', 1)
# reply_data['lable'] = reply_data['lable'].astype(int)

In [ ]:
# 욕설 데이터 불러오기
f = open('badwords/badwords_data.txt', 'r', encoding = "utf-8")
bad_words = []

lines = f.readlines()
for line in lines:
    line = line.strip()
    if not line: break
    bad_words.append(line)
f.close()

In [ ]:
f = open('C://Users\chldl\Desktop\SWtube\CodeSpace\Beautiful-Internet-world\Twitch\chat\worlds_lck_1.txt', 'r', encoding = "utf-8")
lck = []

lines = f.readlines()
for line in lines:
    line = line.strip()
    if not line: break
    lck.append(line)
f.close()

lck_chat = []
for i in range(0, len(lck)):
    temp = lck[i].rfind(':')
    real_text = lck[i][temp + 2:]
    lck_chat.append(real_text)

In [ ]:
import warnings
warnings.filterwarnings(action = 'ignore')
from konlpy.tag import Okt # 형태소 분석에 사용할 konlpy 패키지의 Okt 클래스를 임포트하고 okt 객체를 생성
okt = Okt()

# 레이블이 달린 댓글 데이터
train_df = pd.read_csv('badwords/reply_data.csv', engine='python', encoding = "utf-8")

# 댓글이 있는 항목만 담기(빈 댓글 삭제)
# text 컬럼이 non-null인 샘플만 train_df에 다시 저장
train_df = train_df[train_df['comment'].notnull()]

# 수정된 train_df의 정보를 다시 확인
print(train_df.info())

# 분류 클래스의 구성을 확인
print(train_df['lable'].value_counts())

In [ ]:
# 한글 외 문자 제거(옵션)
import re # 정규식을 사용하기 위해 re 모듈을 임포트
# ‘ㄱ ~‘힣’까지의 문자를 제외한 나머지는 공백으로 치환, 영문: a-z| A-Z
train_df['comment'] = train_df['comment'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
print(train_df.head())

# Train용 데이터셋의 정보를 재확인
print(train_df.info())
text = train_df['comment'] # 시리즈 객체로 저장
score = train_df['lable']

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(text, score , test_size=0.2, random_state=0)
print(len(train_x), len(train_y), len(test_x), len(test_y))

In [ ]:
# 3분 정도 소요됩니다.
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
print(tfv_train_x)

In [ ]:
from sklearn.linear_model import LogisticRegression # 이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

In [ ]:
tfv_test_x = tfv.transform(test_x)
# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)
test_predict = grid_cv.best_estimator_.predict(tfv_test_x)
from sklearn.metrics import accuracy_score
print('혐오 표현 분석 모델의 정확도 : ',round(accuracy_score(test_y, test_predict), 3))

In [ ]:
input_text = '최인열은 쓰레기같고 인간 말종이다.'

#입력 텍스트에 대한 전처리 수행
input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
input_text = [" ".join(input_text)]

# 입력 텍스트의 피처 벡터화
st_tfidf = tfv.transform(input_text)

# 최적 혐오표현 분석 모델에 적용하여 혐오표현 평가
st_predict = grid_cv.best_estimator_.predict(st_tfidf)

#예측 결과 출력
if(st_predict == 1):
    print("혐오표현")
else:
    print("혐오표현이 아님")

In [ ]:
# 분석할 타겟 데이터 불러오기
taxi = pd.read_excel('data/hanmunchul_taxi.xlsx')

In [ ]:
hate = 0
error = 0
hate_data = []

for i in range(0, len(taxi)):
    try:
        if (i % 500 == 0):
            print(input_text, st_tfidf)
        input_text = taxi['comment'][i]

        #입력 텍스트에 대한 전처리 수행
        input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(input_text))
        input_text = [" ".join(input_text)]

        # 입력 텍스트의 피처 벡터화
        st_tfidf = tfv.transform(input_text)

        # 최적 혐오표현 모델에 적용하여 혐오표현 평가
        st_predict = grid_cv.best_estimator_.predict(st_tfidf)

        #예측 결과 출력
        if(st_predict == 1):
            if (i % 500 == 0):
                hate_data.append(input_text)
            hate += 1
        else:
            continue
    except:
        error += 1
        pass

In [ ]:
# 욕설이 포함된 댓글 비율 분석
bad_words_count = 0

for i in range(0, len(taxi)):
    input_text = taxi['comment'][i]

    #입력 텍스트에 대한 전처리 수행
    input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(input_text))
    input_text = [" ".join(input_text)]

    if(any(bad_word in str(input_text) for bad_word in bad_words)):
        print(input_text)
        bad_words_count += 1

In [ ]:
print("욕설이 포함된 댓글 : ", round(bad_words_count / len(taxi) * 100, 3), "%")
print("모델이 혐오 표현으로 분류한 댓글 : ", round(hate / len(taxi) * 100, 3), "%")

In [ ]:
hate = 0
error = 0
hate_data = []

for i in range(0, len(lck_chat)):
    try:
        if (i % 500 == 0):
            print(input_text, st_tfidf)
        input_text = lck_chat[i]

        #입력 텍스트에 대한 전처리 수행
        input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(input_text))
        input_text = [" ".join(input_text)]

        # 입력 텍스트의 피처 벡터화
        st_tfidf = tfv.transform(input_text)

        # 최적 혐오표현 모델에 적용하여 혐오표현 평가
        st_predict = grid_cv.best_estimator_.predict(st_tfidf)

        #예측 결과 출력
        if(st_predict == 1):
            if (i % 500 == 0):
                hate_data.append(input_text)
            hate += 1
        else:
            continue
    except:
        error += 1
        pass

In [ ]:
print("욕설이 포함된 댓글 : ", round(bad_words_count / len(lck_chat) * 100, 3), "%")
print("모델이 혐오 표현으로 분류한 댓글 : ", round(hate / len(lck_chat) * 100, 3), "%")

In [ ]:
# 욕설이 포함된 댓글 비율 분석
bad_words_count = 0

for i in range(0, len(lck_chat)):
    input_text = lck_chat[i]

    #입력 텍스트에 대한 전처리 수행
    input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(input_text))
    input_text = [" ".join(input_text)]

    if(any(bad_word in str(input_text) for bad_word in bad_words)):
        print(input_text)
        bad_words_count += 1

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, test_predict))

In [55]:
import glob
import os

path = glob.glob('data/*')
data = []
for i in range(0, len(path)):
    data.append(os.path.basename(path[i]))

reply_result = pd.DataFrame({"Video" : data, 'bad_words': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'hate': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]})

In [56]:
error = 0
hate_data = []

for i in range(0, len(data)):
    hate = 0
    bad_words_count = 0
    reply = pd.read_excel('data/{}'.format(data[i]))

    for j in range(0, len(reply)):
        try:
            input_text = reply['comment'][j]

            #입력 텍스트에 대한 전처리 수행
            input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(input_text))
            input_text = [" ".join(input_text)]

            if(any(bad_word in str(input_text) for bad_word in bad_words)):
              bad_words_count += 1

            # 입력 텍스트의 피처 벡터화
            st_tfidf = tfv.transform(input_text)

            # 최적 혐오표현 모델에 적용하여 혐오표현 평가
            st_predict = grid_cv.best_estimator_.predict(st_tfidf)

            #예측 결과 출력
            if(st_predict == 1):
              hate_data.append(input_text)
              hate += 1
            else:
              continue
        except:
           error += 1
           pass

    reply_result.loc[i, 'bad_words'] = round(bad_words_count / len(reply) * 100, 3)
    reply_result.loc[i, 'hate'] = round(hate / len(reply) * 100, 3)
    hate_data.append("영상이바뀝니다" + data[i] + "영상이바뀝니다")
    print(reply_result)

                                              Video  bad_words  hate
0                             Cr-9RqDFy8s_쇼메이커.xlsx      5.833  37.5
1                              hanmunchul_benz.xlsx      0.000   0.0
2                              hanmunchul_bike.xlsx      0.000   0.0
3                              hanmunchul_taxi.xlsx      0.000   0.0
4                 JPtAxtvGKDo_여성할당제 토론하다 싸움날 뻔.xlsx      0.000   0.0
5                        LCK 결승전 T1 VS 담원 set2.xlsx      0.000   0.0
6                       LCK 결승전 T1 VS 담원 set4 .xlsx      0.000   0.0
7                                 man_vs_woman.xlsx      0.000   0.0
8                                      son_bad.xlsx      0.000   0.0
9                              TCzueqtnQUE_페이커.xlsx      0.000   0.0
10  xJIBg83BbF4_말도 안되는 갤럭시 S21 울트라 VS 3년 쓴 아이폰.xlsx      0.000   0.0
                                              Video  bad_words    hate
0                             Cr-9RqDFy8s_쇼메이커.xlsx      5.833  37.500
1                             

In [57]:
reply_result

,Video,bad_words,hate
0,Cr-9RqDFy8s_쇼메이커.xlsx,5.833,37.500
1,hanmunchul_benz.xlsx,11.668,50.211
2,hanmunchul_bike.xlsx,7.412,46.168
3,hanmunchul_taxi.xlsx,9.645,41.008
4,JPtAxtvGKDo_여성할당제 토론하다 싸움날 뻔.xlsx,14.950,62.125
5,LCK 결승전 T1 VS 담원 set2.xlsx,6.497,45.244
6,LCK 결승전 T1 VS 담원 set4 .xlsx,5.944,39.759
7,man_vs_woman.xlsx,8.326,50.042
8,son_bad.xlsx,4.454,28.161
9,TCzueqtnQUE_페이커.xlsx,8.051,45.763


In [ ]:
hate_text = open('hate_text', 'w', encoding='UTF-8')

for i in range(0, len(hate_data)):
    input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(str(hate_data[i]))
    input_text = [" ".join(input_text)]
    hate_text.write(input_text[0] + "\n")

hate_text.close()

In [1]:
f = open('hate_text.txt', 'r', encoding = "utf-8")
bad_words = []

lines = f.readlines()
for line in lines:
    line = line.strip()
    if not line: break
    bad_words.append(line)
f.close()

In [16]:
import random

find_berry = '영상이바뀝니다'
index = [i for i in range(len(bad_words)) if find_berry in bad_words[i]]
print(index)

index.insert(0, 0)

sample = []
for i in range(0, len(index) - 1):
    temp = []
    temp = bad_words[index[i]:index[i+1]]
    temp = random.sample(temp, 3)
    for j in range(0, len(temp)):
        sample.append(temp[j])

print(sample)

[45, 8119, 11060, 31614, 41609, 41805, 42301, 42897, 43094, 43203, 43348]
['이런 쇼메랑 점 가까이 차이나는 쪼말은 진짜뭐지', '팬코 자제좀하셈', '김밥오이빼고는 왜 저기 있는건데ㅋㅋㅋㅋ', '나여잔데 블박님 덩치 부럽네요ㅠ 저는 이에 초딩몸뚱아리라 제가내리면 쌩지랄했을걸요ㅠㅠ 진짜 부럽네요 그리고 저런놈 참교육 제대로 시켜주셔서 제가 더감사', '양아치내 ㅉㅉ', '저게 진짜 말도 안되는게 결국 끼워줬는데 저랬다는게 어이없음 레이 차주분은 보복 당할만한 일을 안했는데 저정도면 초보 운전자나 여성 운전자였다면서 크게 사고나거나 폭행까지 갔을듯 사고나라는 식으로 운전하는데 맘같아선 살인미수다 저 상황에서도 침착하게 현명한 판단하신 레이 차주분이 대단하시네요', '진짜 교통사고 과실비율 이거 바꿔야함 정상적으로 신호받고 진행하는데 신호위반 해서 들어와서 박은걸 이라고 우기는 보험사 놈들 웃기는 개소리 하고 자빠졌어', '저거 배달대행 오토바이 아닌가요 요즘 저런 잡것들 전부다 신호위반이던데 ㅡㅡ', '안죽었데요 왜 안죽었데요', '아니 택시운전사 너무하네요 무슨 응급구조사 없다고 그렇게 극단적으로 결정하는 게 어디 있습니까 그 며느님은 얼마나 서러우셨을 거에요 정말 화가 나네요', '택시인생 죠졋네 ㅂㅅ', '탹시야 너이제 어쩔래 너도 이제 실려가는거냐 딱보니 앞에 끼우기 싫어서 사고낸거 다안다 사람죽엇으니 넌 어찌 책임질려고 지랄햇누 참 지랄병 만만세다 ㅋㅋ그냥 집에서 나오지마쇼', '비율이 낮은게 문제면 높이는게 다가 아니고 높아지게끔 도와야 맞는거 아닌가 그렇게 자신들이 없나', '여성기업 만들어 얼마나 잘되나보게 여가부는 성불평등 집단아니야', '응 닥치고 군대부터 반반씩 가자 시벌것들 물론 사병지원이지 꿀빨생각말고 좃같이 년동안 구르고 나이처묵고 무좀걸리고 요구하자 취업하면 요일지정해서 당직질 할생각말고 월급루팡년들아', '오히려 치욕인데 섬머결승에서 솔랭식 밴픽 이건 그냥 쇼메 킬주려는걸로 

In [5]:
import pandas as pd
data1 = pd.read_csv('아름다운 인터넷 세상 혐오표현 설문(응답) - 설문지 응답 시트1.csv')


data1['상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식)']

0     010-7704-8957
1     010-9491-5530
2     010-2707-7711
3     010-7136-5742
4     010-6880-4828
5     010-7935-3471
6     010-9006-8780
7     010-8421-0413
8     010-2713-9461
9     010-6207-4907
10    010-6225-1169
11    010-7906-0459
12    010-8377-3220
13    010-9916-6459
14    010-9085-7278
15    010-7520-2133
16    010-8220-4131
17    010-9884-9867
18    010-4058-3799
19    010-3849-8591
20    010-3495-1772
21    010-5249-7646
22    010-2278-6072
23    010-2362-4878
24    010-5115-8762
25    010-9327-7735
26    010-2894-2546
27    010-6267-7681
28    010-9137-4426
29    010-4275-6106
Name: 상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식), dtype: object

In [8]:
good = data1.sample(10)
good['상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식)']

25    010-9327-7735
26    010-2894-2546
24    010-5115-8762
0     010-7704-8957
16    010-8220-4131
19    010-3849-8591
7     010-8421-0413
14    010-9085-7278
28    010-9137-4426
17    010-9884-9867
Name: 상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식), dtype: object

In [10]:
good2 = data1.sample(2)
good2['상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식)']

23    010-2362-4878
12    010-8377-3220
Name: 상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식), dtype: object

In [12]:
good3 = data1.sample(1)
good3['상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식)']

1    010-9491-5530
Name: 상품을 전달할 수 있도록 전화번호를 주시면 감사하겠습니다. (010-XXXX-XXXX의 형식), dtype: object